In [1]:
from selenium import webdriver
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [2]:
df = pd.read_csv('nasdaq_stocks.csv')
stock_symbols = list(df['Symbol'])
driver = webdriver.Chrome()

shares_number, volume, net_income, dividends, employee_amount = [], [], [], [], []
previous_close, analysts, price_target, earnings_date = [], [], [], []

base_url = 'https://stockanalysis.com/stocks/{}/'

for symbol in stock_symbols:
    if pd.isna(symbol):
        shares_number.append(np.nan)
        net_income.append(np.nan)
        dividends.append(np.nan)
        volume.append(np.nan)
        previous_close.append(np.nan)
        analysts.append(np.nan)
        price_target.append(np.nan)
        earnings_date.append(np.nan)
        employee_amount.append(np.nan)
        continue

    driver.get(base_url.format(symbol.lower())) 
    
    try:
        tables = driver.find_elements(By.XPATH, '//*[@class="w-[48%] text-sm text-gray-900 dark:text-dark-200 tiny:text-small lg:w-auto lg:min-w-[210px]"]')
        tds_left_table = tables[0].find_elements(By.TAG_NAME, 'td')
        shares_number.append(tds_left_table[7].text)
        net_income.append(tds_left_table[5].text)
        dividends.append(tds_left_table[15].text.split()[0])
        tds_right_table = tables[1].find_elements(By.TAG_NAME, 'td')
        volume.append(tds_right_table[1].text)
        previous_close.append(tds_right_table[5].text)
        analysts.append(tds_right_table[13].text)
        price_target.append(tds_right_table[15].text.split()[0])
        earnings_date.append(tds_right_table[17].text)
        employees = driver.find_elements(By.CSS_SELECTOR, '.dothref.text-default')[2].text
        employee_amount.append(employees)
        
    except Exception as e:
        shares_number.append(np.nan)
        net_income.append(np.nan)
        dividends.append(np.nan)
        volume.append(np.nan)
        previous_close.append(np.nan)
        analysts.append(np.nan)
        price_target.append(np.nan)
        earnings_date.append(np.nan)
        employee_amount.append(np.nan)
        
driver.quit()

In [ ]:
data = {
    'Shares Number': shares_number,
    'Volume': volume,
    'Net Income': net_income,
    'Dividends': dividends,
    'Employee Amount': employee_amount,
    'Previous Close': previous_close,
    'Analysts': analysts,
    'Price Target': price_target,
    'Earnings Date': earnings_date
}

stocks_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
stocks_df.to_csv('nasdaq_stock_data.csv', index=False)

print("Data saved to nasdaq_stock_data.csv")